<h1 style='text-align: center'>SQL Queries</h1>

## Getting Data From A SQL Database

### The Structure of a SQL Query

<img src='images/sql_statement.jpg'/>

#### GROUP BY

- Group columns by similar values
- SELECT COUNT(id), city from students GROUP BY city

#### HAVING

- Use to apply filter AFTER a `GROUP BY` based on aggregate criteria 
- `WHERE` is applied for conditions prior to the `GROUP BY`, `HAVING` is applied afterwards

For example, if we had a table of student names and the courses they were taking, we could ask a question such as which classes have 3 or more students with the name Matt?

Such a query would look something like this:

```SQL
SELECT
  class,
  COUNT(student_name) AS number_of_matts
FROM student_courses
WHERE student_name = "Matt"
GROUP BY 1
HAVING COUNT(student_name) >= 3;
```

In [1]:
import sqlite3

conn = sqlite3.connect('tutorial.db')
c = conn.cursor()

In [2]:
columns = [x[0] for x in c.execute('select * from students').description]

In [3]:
columns

['name',
 'birthdate',
 'siblings',
 'birth_place',
 'years_in_nyc',
 'favorite_food']

### Questions
1. What are the names of all of the students?
2. Which student has the most siblings?
3. How many students are only children?
4. Which 3 students have lived in NYC the shortest amount of time?
5. How many students are native New Yorkers?
6. Do any two students have the same favorite food?


In [17]:
import pandas as pd
pd.read_sql_query('''SELECT * FROM STUDENTS LIMIT 3''', conn)

,name,birthdate,siblings,birth_place,years_in_nyc,favorite_food
0,Sean Abu Wilson,02/06,2,"Birmingham, AL",7.60,guacamole
1,David Miller,06/06,2,"New York, NY",25.75,pizza
2,Abhijeet Kamble,05/07,0,Wardha India,0.50,Biriyani


1. What are the names of all of the students.

In [18]:
# What are the names of all of the students?
import pandas as pd
pd.read_sql_query('''SELECT name FROM STUDENTS''', conn)

,name
0,Sean Abu Wilson
1,David Miller
2,Abhijeet Kamble
3,Samantha Jackson
4,Anmol Srivats
5,Ran Tokman
6,Amy Li
7,Florencia Leoni
8,Austin Krause
9,Natalie Overchuk


2. Which student has the most siblings?

> This is great place to use a subquery. Encourage students who are initially struggling with a question along the lines of "How could you select the largest number of siblings that anyone has in the group?" From there, you can further push students with a hint if needed: "How can you now make a selection using the result of this, [embedded as a subquery]?"

In [27]:
# Which student has the most siblings?
pd.read_sql_query('''
SELECT name, siblings 
FROM students 
WHERE siblings = (SELECT MAX(siblings) FROM students)''', conn)

,name,siblings
0,Florencia Leoni,4
1,Mohamad Eldebek,4
2,Menachi Korn,4
3,Miguel Peña,4


3. How many students are only children?

In [28]:
# How many students are only children?
pd.read_sql_query('''
SELECT count(*) num_of_only_child
FROM students 
WHERE siblings = 0''', conn)

,num_of_only_child
0,3


4. Which 3 students have lived in NYC the shortest amount of time? (How long has each lived in NYC?)
    

In [29]:
# Which 3 students have lived in NYC the shortest amount of time?
pd.read_sql_query('''
SELECT name, years_in_nyc
FROM students 
ORDER BY 2 
LIMIT 3
''', conn)


,name,years_in_nyc
0,Anmol Srivats,0.05
1,Natalie Overchuk,0.10
2,Austin Krause,0.17


5. How many students are native New Yorkers?

In [35]:
# How many students are native New Yorkers?
pd.read_sql_query('''
SELECT COUNT(*) num_of_native_NY_students
FROM students 
WHERE LOWER(TRIM(birth_place)) LIKE '%ny'
''', conn)


,num_of_native_NY_students
0,6


6. Do any two students have the same favorite food?

This problem employs the `Having` clause.  Be sure to review the difference between the where and having clause here. (Where filters apply before the group by clause and conditions following the having clause are filters applied after the group by on the resulting aggregate [statistics].) A useful example in doing so, could be to modify the question to something with an additional filtering criterion such as 'do any native new yorkers have the same favorite food?' This would force students to use a where clause prior to the group by to filter the results. Alternatively, see the question below for an alternative but related problem on favorite foods.

In [14]:
pd.read_sql_query("""
SELECT favorite_food, count(favorite_food)
FROM students
GROUP BY favorite_food
HAVING count(favorite_food) > 1
""", conn)

[('pizza', 2), ('steak', 2)]

In [38]:
pd.read_sql_query("""
SELECT 
    s1.name,
    s2.name,
    favorite_food
    
FROM students s1
LEFT JOIN students s2 USING(favorite_food)
WHERE s1.name != s2.name

""", conn)

,name,name,favorite_food
0,David Miller,Akshay Sharma,pizza
1,Florencia Leoni,Nicole Roach,steak
2,Akshay Sharma,David Miller,pizza
3,Nicole Roach,Florencia Leoni,steak


## More Questions

What are the favorite foods of this classroom?

In [39]:
pd.read_sql_query('''
SELECT favorite_food, count(favorite_food)
FROM students 
GROUP BY 1
ORDER BY 2 DESC
LIMIT 4
''', conn)

,favorite_food,count(favorite_food)
0,pizza,2
1,steak,2
2,Avocado,1
3,Biriyani,1


7. Which student was born closest to the cohort's graduation date?

In [46]:
pd.read_sql_query('''
SELECT name, birthdate
FROM students 
ORDER BY ('12/06' - birthdate)
''', conn)

,name,birthdate
0,Amy Li,12/29
1,Maks Pazuniak,12/19
2,Alex Mitrani,12/06
3,Anmol Srivats,11/19
4,Yish Lim,11/21
5,Austin Krause,10/22
6,Samantha Jackson,08/04
7,Miguel Peña,08/14
8,Fhel Dimaano,08/20
9,David Miller,06/06


In [48]:
pd.read_sql_query('''
SELECT name, 
MIN(ABS(julianday('2019'||'-'||substr(birthdate, 1, 2)||'-'||substr(birthdate, 4, 2)) - julianday('2019-12-06')))
AS days
FROM students 
ORDER BY 2 DESC
''', conn)

,name,days
0,Alex Mitrani,0.0


In [ ]:
# MySQL: concat and date functions

# MySQL version

# SELECT name,
# abs(date(concat('2019-',substr(birthdate,1,2), '-', substr(birthdate,4,2))) - date('2019-08-26')) AS days
# FROM students
# ORDER BY 2
# LIMIT 1